In [1]:
import os
import torch
import torch.nn as nn 
from torch.nn import functional as F
from utils.common import setup_seed 
from MLPN.loader import init_dataset_train
from utils.loader import  init_dataset_test, environments
from utils.metrics import metrics
from MLPN.model import CSWinTransv2_threeIn
from MLPN.utils import extract_feature, get_id, extract_feature, SAM, SupConLoss, one_LPN_output
from torch.autograd import Variable

from torch.optim import lr_scheduler
from tqdm.contrib import tzip

from utils.competition import get_result_rank10, get_SatId_160k
from LPN.image_folder_ import CustomData160k_drone, CustomData160k_sat
from torchvision import datasets, transforms
from torchvision.transforms import InterpolationMode
import numpy as np
from tqdm import tqdm
import tarfile
import zipfile



def get_fixed_dataloader(dataloaders, style='mixed',num=None):
    dataloader_drone = list()
    dataloader_sat = list()
    cnt = 0
    for data, data3 in tzip(dataloaders['drone'], dataloaders['satellite']):
        dataloader_drone.append(data)
        dataloader_sat.append(data3)
        cnt = cnt + 1
        if type(num) is int and cnt==num:
            break
    print('done synthesis {} style image'.format(style))
    return dataloader_drone, dataloader_sat




c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir=None
style='mixed'
model_name='net'
num_epochs=210
lr = 0.005
batchsize = 8
block = 4
checkpoint_interval = 10
checkpoint_start = 0
droprate = 0.75
update_aug_img = [40, 80, 120, 160, 180, 200]
num_worker_imgaug = 0


data_dir = os.path.join(os.getcwd(), 'University-Release', 'train')
image_datasets, dataloaders, dataset_sizes = init_dataset_train(data_dir, batchsize=batchsize, style=style, num_worker_imgaug=num_worker_imgaug)
dataloader_drone, dataloader_sat = get_fixed_dataloader(dataloaders)


c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
  0%|          | 0/4732 [00:01<?, ?it/s]


UnboundLocalError: local variable 'transforms_d' referenced before assignment